In [1]:
from sklearn.datasets import load_iris
from sklearn import ensemble
import os
os.chdir('C:/test/data')
import pandas as pd
df = pd.read_csv('diabetes.csv')

y = df.pop('Outcome')

In [30]:
clf = ensemble.GradientBoostingClassifier(subsample=0.8)
clf.fit(df,y)

clf.n_estimators_,len(clf.oob_improvement_)

for each in clf.staged_predict_proba(df.iloc[0:2,:]):
    print(each)

y[:2]

In [3]:
import xgboost as xgb
import lightgbm as lgb

In [4]:
xclf = xgb.XGBClassifier()
lclf = lgb.LGBMClassifier(num_leaves=10)

In [5]:
xclf.fit(df,y)
lclf.fit(df,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=10, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [7]:
x = xclf.get_booster()
data = xgb.DMatrix(df)
x.get_dump()[3]
booster = lclf.booster_
model = booster.dump_model()
tree_infos = model['tree_info']
tree_info = tree_infos[3]

In [9]:
tree = x.get_dump()[0]
tree

'0:[Glucose<128] yes=1,no=2,missing=1\n\t1:[Age<29] yes=3,no=4,missing=3\n\t\t3:[BMI<45.4000015] yes=7,no=8,missing=7\n\t\t\t7:leaf=-0.167527676\n\t\t\t8:leaf=0.0500000007\n\t\t4:[BMI<26.3500004] yes=9,no=10,missing=9\n\t\t\t9:leaf=-0.164444447\n\t\t\t10:leaf=-0.0395480208\n\t2:[BMI<29.9500008] yes=5,no=6,missing=5\n\t\t5:[Glucose<146] yes=11,no=12,missing=11\n\t\t\t11:leaf=-0.12888889\n\t\t\t12:leaf=0.00512820529\n\t\t6:[Glucose<158] yes=13,no=14,missing=13\n\t\t\t13:leaf=0.0420168079\n\t\t\t14:leaf=0.141666666\n'

In [12]:
from sklearn.tree import DecisionTreeClassifier,export_text

In [14]:
model = DecisionTreeClassifier(max_depth=3)
model.fit(df,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [15]:
export_text(model)

'|--- feature_1 <= 127.50\n|   |--- feature_7 <= 28.50\n|   |   |--- feature_5 <= 45.40\n|   |   |   |--- class: 0\n|   |   |--- feature_5 >  45.40\n|   |   |   |--- class: 1\n|   |--- feature_7 >  28.50\n|   |   |--- feature_5 <= 26.35\n|   |   |   |--- class: 0\n|   |   |--- feature_5 >  26.35\n|   |   |   |--- class: 0\n|--- feature_1 >  127.50\n|   |--- feature_5 <= 29.95\n|   |   |--- feature_1 <= 145.50\n|   |   |   |--- class: 0\n|   |   |--- feature_1 >  145.50\n|   |   |   |--- class: 1\n|   |--- feature_5 >  29.95\n|   |   |--- feature_1 <= 157.50\n|   |   |   |--- class: 1\n|   |   |--- feature_1 >  157.50\n|   |   |   |--- class: 1\n'

(1, '<=', 123.50000000000001)
(7, '<=', 28.500000000000004)
(4, '<=', 144.50000000000003)
(4, '<=', 22.500000000000004)
(5, '<=', 26.450000000000003)
(1, '<=', 154.50000000000003)
(7, '<=', 24.500000000000004)
(5, '<=', 29.950000000000003)
(6, '<=', 0.4015000000000001)


In [31]:
tree_info['tree_structure']

{'split_index': 0,
 'split_feature': 1,
 'split_gain': 74.63819885253906,
 'threshold': 123.50000000000001,
 'decision_type': '<=',
 'default_left': True,
 'missing_type': 'None',
 'internal_value': 0,
 'internal_count': 768,
 'left_child': {'split_index': 2,
  'split_feature': 7,
  'split_gain': 15.515999794006348,
  'threshold': 28.500000000000004,
  'decision_type': '<=',
  'default_left': True,
  'missing_type': 'None',
  'internal_value': -0.603679,
  'internal_count': 446,
  'left_child': {'leaf_index': 0,
   'leaf_value': -0.0975887674959255,
   'leaf_count': 254},
  'right_child': {'split_index': 3,
   'split_feature': 4,
   'split_gain': 11.591699600219727,
   'threshold': 144.50000000000003,
   'decision_type': '<=',
   'default_left': True,
   'missing_type': 'None',
   'internal_value': -0.158439,
   'internal_count': 192,
   'left_child': {'split_index': 5,
    'split_feature': 4,
    'split_gain': 8.160120010375977,
    'threshold': 22.500000000000004,
    'decision_type'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import convolve
from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone


# #############################################################################
# Setting up

def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    def shift(x, w):
        return convolve(x.reshape((8, 8)), 
                        mode='constant', weights=w).ravel()

    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y


# Load Data
digits = datasets.load_digits()
X = np.asarray(digits.data, 'float32')
X, Y = nudge_dataset(X, digits.target)
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

# Models we will use
logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1,
                                           multi_class='multinomial')
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

# Training
# Hyper-parameters. These were set by cross-validation,
# using a GridSearchCV. Here we are not performing cross-validation to
# save time.
rbm.learning_rate = 0.06
rbm.n_iter = 20
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 6000

# Training RBM-Logistic Pipeline
rbm_features_classifier.fit(X_train, Y_train)

# Training the Logistic regression classifier directly on the pixel
raw_pixel_classifier = clone(logistic)
raw_pixel_classifier.C = 100.
raw_pixel_classifier.fit(X_train, Y_train)

# Evaluation

Y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))

Y_pred = raw_pixel_classifier.predict(X_test)
print("Logistic regression using raw pixel features:\n%s\n" % (
    metrics.classification_report(Y_test, Y_pred)))


plt.figure(figsize=(4.2, 4))
for i, comp in enumerate(rbm.components_):
    plt.subplot(10, 10, i + 1)
    plt.imshow(comp.reshape((8, 8)), cmap=plt.cm.gray_r,
               interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
plt.suptitle('100 components extracted by RBM', fontsize=16)
plt.subplots_adjust(0.08, 0.02, 0.92, 0.85, 0.08, 0.23)

plt.show()

In [10]:
'''the binary tree structure;
the depth of each node and whether or not it’s a leaf;
the nodes that were reached by a sample using the decision_path method;
the leaf that was reached by a sample using the apply method;
the rules that were used to predict a sample;
the decision path shared by a group of samples.'''

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

estimator = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
estimator.fit(X_train, y_train)

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

node_indicator = estimator.decision_path(X_test)

# Similarly, we can also have the leaves ids reached by each sample.

leave_id = estimator.apply(X_test)

sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] == node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision id node %s : (X_test[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

# For a group of samples, we have the following common node.
sample_ids = [0, 1]
common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                len(sample_ids))

common_node_id = np.arange(n_nodes)[common_nodes]

print("\nThe following samples %s share the node %s in the tree"
      % (sample_ids, common_node_id))
print("It is %s %% of all nodes." % (
    100 * len(common_node_id) / n_nodes,))

The binary tree structure has 5 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 3] <= 0.800000011920929 else to node 2.
	node=1 leaf node.
	node=2 test node: go to node 3 if X[:, 2] <= 4.950000047683716 else to node 4.
		node=3 leaf node.
		node=4 leaf node.

Rules used to predict sample 0: 
decision id node 0 : (X_test[0, 3] (= 2.4) > 0.800000011920929)
decision id node 2 : (X_test[0, 2] (= 5.1) > 4.950000047683716)

The following samples [0, 1] share the node [0 2] in the tree
It is 40.0 % of all nodes.


In [12]:
estimator.get_depth()

2

In [16]:
clf.n_outputs_

1